# Background Data Extraction for Site Analysis

This notebook extracts background environmental data for any Area of Interest (AOI):
- **AAFC Annual Crop Inventory**: Historical crop types for each year
- **ERA-5 Land Precipitation**: Monthly precipitation with anomaly analysis

Simply upload your AOI polygon, specify the date range, and export the combined dataset as CSV for use in other analyses.

## 1. Setup and Authentication

In [ ]:
# Import required libraries
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML
import json
import os
from io import BytesIO
import zipfile
import calendar
from datetime import datetime
import geopandas as gpd
import fiona

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Initialize Earth Engine
try:
    ee.Initialize()
    print("✓ Earth Engine initialized successfully")
except:
    print("Authenticating with Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="jessemapping")  # Replace with your GEE project ID
    print("✓ Earth Engine initialized successfully")

## 2. AOI Upload and Configuration

In [ ]:
# Global variables for storing geometries
aoi_geometry = None
uploaded_filename = None

def parse_kml(content):
    """Parse KML content and extract geometry"""
    try:
        import xml.etree.ElementTree as ET
        root = ET.fromstring(content)
        
        # Handle KML namespaces
        namespaces = {'kml': 'http://www.opengis.net/kml/2.2'}
        
        # Find all coordinates
        coords_elements = root.findall('.//kml:coordinates', namespaces)
        if not coords_elements:
            # Try without namespace
            coords_elements = root.findall('.//coordinates')
        
        if coords_elements:
            coords_text = coords_elements[0].text.strip()
            # Parse coordinates (format: lon,lat,alt or lon,lat)
            coords = []
            for point in coords_text.split():
                parts = point.split(',')
                if len(parts) >= 2:
                    coords.append([float(parts[0]), float(parts[1])])
            return ee.Geometry.Polygon([coords])
    except Exception as e:
        print(f"Error parsing KML: {e}")
        return None

def parse_geojson(content):
    """Parse GeoJSON content and extract geometry"""
    try:
        geojson_dict = json.loads(content)
        
        # Handle different GeoJSON structures
        if geojson_dict.get('type') == 'FeatureCollection':
            # Get first feature
            if geojson_dict.get('features'):
                feature = geojson_dict['features'][0]
                return ee.Geometry(feature['geometry'])
        elif geojson_dict.get('type') == 'Feature':
            return ee.Geometry(geojson_dict['geometry'])
        elif geojson_dict.get('type') in ['Polygon', 'MultiPolygon', 'Point']:
            return ee.Geometry(geojson_dict)
    except Exception as e:
        print(f"Error parsing GeoJSON: {e}")
        return None

def parse_shapefile(file_contents, filename):
    """Parse shapefile (from zip) and extract geometry"""
    try:
        # Save uploaded file temporarily
        import tempfile
        import shutil
        
        with tempfile.TemporaryDirectory() as tmpdir:
            # Handle ZIP file
            if filename.endswith('.zip'):
                with zipfile.ZipFile(BytesIO(file_contents)) as zf:
                    zf.extractall(tmpdir)
                
                # Find the .shp file
                shp_files = [f for f in os.listdir(tmpdir) if f.endswith('.shp')]
                if not shp_files:
                    print("No .shp file found in the ZIP archive")
                    return None
                
                shp_path = os.path.join(tmpdir, shp_files[0])
            else:
                # Single .shp file
                shp_path = os.path.join(tmpdir, filename)
                with open(shp_path, 'wb') as f:
                    f.write(file_contents)
            
            # Read with geopandas
            gdf = gpd.read_file(shp_path)
            
            # Convert to Earth Engine geometry
            # Get the first geometry
            geom = gdf.geometry.iloc[0]
            
            # Convert to GeoJSON then to Earth Engine
            geojson = geom.__geo_interface__
            return ee.Geometry(geojson)
            
    except Exception as e:
        print(f"Error parsing shapefile: {e}")
        return None

def handle_upload(change):
    """Handle file upload and parse geometry"""
    global aoi_geometry, uploaded_filename
    
    if change['new']:
        uploaded_file = change['new'][0]
        filename = uploaded_file['name']
        content = uploaded_file['content']
        
        uploaded_filename = filename
        print(f"Processing: {filename}")
        
        # Parse based on file type
        if filename.endswith('.kml'):
            aoi_geometry = parse_kml(content.decode('utf-8'))
        elif filename.endswith('.geojson') or filename.endswith('.json'):
            aoi_geometry = parse_geojson(content.decode('utf-8'))
        elif filename.endswith('.zip') or filename.endswith('.shp'):
            aoi_geometry = parse_shapefile(content, filename)
        else:
            print(f"Unsupported file type: {filename}")
            return
        
        if aoi_geometry:
            # Calculate area
            area = aoi_geometry.area().divide(10000).getInfo()  # Convert to hectares
            print(f"✓ AOI loaded successfully!")
            print(f"  Area: {area:.2f} hectares")
            
            # Get centroid for display
            centroid = aoi_geometry.centroid().coordinates().getInfo()
            print(f"  Centroid: {centroid[1]:.5f}°N, {centroid[0]:.5f}°W")
            
            # Update status
            status_output.clear_output()
            with status_output:
                display(HTML(f'<div style="color: green; font-weight: bold;">✓ AOI loaded: {area:.1f} ha</div>'))
        else:
            status_output.clear_output()
            with status_output:
                display(HTML('<div style="color: red;">⚠ Failed to parse geometry</div>'))

In [ ]:
# Create upload widget
upload_widget = widgets.FileUpload(
    accept='.kml,.geojson,.json,.zip,.shp',
    multiple=False,
    description='Upload AOI:',
    button_style='success'
)

# Status output
status_output = widgets.Output()

# Connect handler
upload_widget.observe(handle_upload, names='value')

# Display widgets
print("Upload your Area of Interest (AOI) polygon:")
print("Supported formats: KML, GeoJSON, Shapefile (SHP/ZIP)")
display(upload_widget)
display(status_output)

## 3. Configure Analysis Parameters

In [ ]:
# Date range configuration
current_year = datetime.now().year

# AAFC date range (available 2009-2023)
aafc_start = widgets.IntSlider(
    value=2017,
    min=2009,
    max=2023,
    step=1,
    description='AAFC Start:',
    style={'description_width': 'initial'}
)

aafc_end = widgets.IntSlider(
    value=2023,
    min=2009,
    max=2023,
    step=1,
    description='AAFC End:',
    style={'description_width': 'initial'}
)

# Precipitation baseline period
precip_baseline_start = widgets.IntSlider(
    value=2010,
    min=1980,
    max=current_year-1,
    step=1,
    description='Baseline Start:',
    style={'description_width': 'initial'}
)

precip_baseline_end = widgets.IntSlider(
    value=2019,
    min=1980,
    max=current_year-1,
    step=1,
    description='Baseline End:',
    style={'description_width': 'initial'}
)

# Analysis years for precipitation
precip_analysis_start = widgets.IntSlider(
    value=2017,
    min=1980,
    max=current_year,
    step=1,
    description='Analysis Start:',
    style={'description_width': 'initial'}
)

precip_analysis_end = widgets.IntSlider(
    value=2023,
    min=1980,
    max=current_year,
    step=1,
    description='Analysis End:',
    style={'description_width': 'initial'}
)

print("Configure Date Ranges:")
print("\nAAFC Crop Inventory:")
display(aafc_start, aafc_end)
print("\nPrecipitation Baseline Period (for calculating 'normal'):")
display(precip_baseline_start, precip_baseline_end)
print("\nPrecipitation Analysis Years:")
display(precip_analysis_start, precip_analysis_end)

## 4. Extract AAFC Crop History

In [ ]:
# AAFC crop classification codes
CROP_CLASSES = {
    10: 'Built-up',
    20: 'Water',
    30: 'Exposed and/or Barren Land',
    34: 'Developed',
    35: 'Shrubland',
    50: 'Grassland',
    80: 'Wetland',
    110: 'Annual Crop (undifferentiated)',
    120: 'Berries',
    121: 'Blueberry',
    122: 'Cranberry',
    130: 'Orchard',
    131: 'Other Orchard',
    132: 'Vineyard',
    133: 'Hops',
    136: 'Other Vegetables',
    137: 'Asparagus',
    139: 'Onions',
    140: 'Potato',
    141: 'Sweet Potato',
    145: 'Sugar Beets',
    146: 'Spring Wheat',
    147: 'Winter Wheat',
    148: 'Wheat (undifferentiated)',
    150: 'Switchgrass',
    151: 'Tobacco',
    152: 'Ginseng',
    153: 'Canola and Rapeseed',
    154: 'Flaxseed',
    155: 'Safflower',
    156: 'Sunflower',
    157: 'Soybeans',
    158: 'Peas',
    160: 'Barley',
    161: 'Oats',
    162: 'Corn',
    163: 'Triticale',
    165: 'Millet',
    166: 'Rye',
    167: 'Spelt',
    174: 'Beans',
    175: 'Fababeans',
    176: 'Lentils',
    177: 'Vegetables (undifferentiated)',
    178: 'Tomatoes',
    179: 'Pumpkins',
    180: 'Squash',
    181: 'Lettuce',
    182: 'Spinach',
    183: 'Beets',
    184: 'Carrots',
    185: 'Cabbage',
    187: 'Peppers',
    188: 'Eggplants',
    189: 'Radishes',
    191: 'Garlic',
    192: 'Sod',
    193: 'Herbs',
    194: 'Nursery',
    195: 'Buckwheat',
    196: 'Canaryseed',
    197: 'Hemp',
    198: 'Vetch',
    199: 'Other Crops',
    200: 'Forest (undifferentiated)',
    210: 'Coniferous',
    220: 'Broadleaf',
    230: 'Mixedwood'
}

def extract_aafc_crop_history(geometry, start_year, end_year):
    """Extract AAFC crop history for the AOI"""
    aafc = ee.ImageCollection('AAFC/ACI')
    crop_history = []
    
    for year in range(start_year, end_year + 1):
        # Get crop inventory for this year
        crop_collection = aafc.filter(ee.Filter.date(f'{year}-01-01', f'{year}-12-31'))
        
        # Check if collection has images
        collection_size = crop_collection.size()
        
        if collection_size.getInfo() > 0:
            crop_img = crop_collection.first()
            # Sample pixels within the geometry
            samples = crop_img.select('landcover').sample(
                region=geometry,
                scale=30,
                numPixels=500,
                seed=year,
                geometries=False
            )
            
            # Get mode (most frequent crop)
            sample_list = samples.aggregate_array('landcover').getInfo()
            
            if sample_list:
                from collections import Counter
                code_counts = Counter(sample_list)
                most_common = code_counts.most_common(3)  # Get top 3 crops
                
                # Primary crop
                primary_code = most_common[0][0]
                primary_count = most_common[0][1]
                primary_pct = (primary_count / len(sample_list)) * 100
                
                crop_history.append({
    'Year': year,
                    'Primary_Crop_Code': primary_code,
                    'Primary_Crop': CROP_CLASSES.get(primary_code, f'Unknown ({primary_code})'),
                    'Primary_Crop_Pct': round(primary_pct, 1),
                    'Secondary_Crop': CROP_CLASSES.get(most_common[1][0], 'None') if len(most_common) > 1 else 'None',
                    'Total_Samples': len(sample_list)
                })
            else:
                crop_history.append({
                    'Year': year,
                    'Primary_Crop_Code': None,
                    'Primary_Crop': 'No Data',
                    'Primary_Crop_Pct': 0,
                    'Secondary_Crop': 'None',
                    'Total_Samples': 0
                })
        else:
            crop_history.append({
                'Year': year,
                'Primary_Crop_Code': None,
                'Primary_Crop': 'No AAFC Data',
                'Primary_Crop_Pct': 0,
                'Secondary_Crop': 'None',
                'Total_Samples': 0
            })
    
    return pd.DataFrame(crop_history)

In [ ]:
# Extract AAFC data
if aoi_geometry:
    print(f"Extracting AAFC crop history ({aafc_start.value}-{aafc_end.value})...")
    aafc_df = extract_aafc_crop_history(aoi_geometry, aafc_start.value, aafc_end.value)
    
    print("\nCrop History:")
    display(aafc_df)
    
    # Visualize crop rotation
    if len(aafc_df) > 0:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        # Get unique crops and assign colors
        unique_crops = aafc_df['Primary_Crop'].unique()
        colors = plt.cm.Set3(np.linspace(0, 1, len(unique_crops)))
        crop_colors = dict(zip(unique_crops, colors))
        
        # Create bar chart
        bars = ax.bar(aafc_df['Year'], [1]*len(aafc_df), 
                      color=[crop_colors[crop] for crop in aafc_df['Primary_Crop']])
        
        # Add crop labels
        for i, (year, crop) in enumerate(zip(aafc_df['Year'], aafc_df['Primary_Crop'])):
            ax.text(year, 0.5, crop[:10], ha='center', va='center', rotation=90, fontsize=9)
        
        ax.set_xlabel('Year', fontsize=12)
        ax.set_ylabel('Crop Type', fontsize=12)
        ax.set_title('Crop Rotation History', fontsize=14, fontweight='bold')
        ax.set_ylim(0, 1)
        ax.set_yticks([])
        
        # Create legend
        legend_elements = [plt.Rectangle((0,0),1,1, fc=crop_colors[crop], label=crop) 
                          for crop in unique_crops]
        ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
        
        plt.tight_layout()
        plt.show()
else:
    print("⚠ Please upload an AOI first")

## 5. Extract ERA-5 Precipitation Data

In [ ]:
def extract_monthly_precipitation(geometry, year, months=None):
    """Extract monthly precipitation from ERA-5 Land"""
    if months is None:
        months = list(range(4, 11))  # April to October
    
    monthly_data = []
    era5_collection = 'ECMWF/ERA5_LAND/MONTHLY_AGGR'
    
    for month in months:
        start_date = f'{year}-{month:02d}-01'
        last_day = calendar.monthrange(year, month)[1]
        end_date = f'{year}-{month:02d}-{last_day}'
        
        # Load ERA5-Land monthly data
        era5_collection_ee = ee.ImageCollection(era5_collection).filter(
            ee.Filter.date(start_date, end_date)
        )
        
        # Check if collection has images
        collection_size = era5_collection_ee.size()
        
        if collection_size.getInfo() > 0:
            era5 = era5_collection_ee.first()
            
            # Extract total precipitation (m to mm conversion)
            precip = era5.select('total_precipitation_sum').multiply(1000)
            
            # Calculate mean precipitation over geometry
            stats = precip.reduceRegion(
                reducer=ee.Reducer.mean(),
                geometry=geometry,
                scale=11132,  # ERA5-Land resolution
                maxPixels=1e9
            )
            
            precip_mm = stats.get('total_precipitation_sum').getInfo()
            
            monthly_data.append({
                'Year': year,
                'Month': month,
                'Month_Name': calendar.month_name[month],
                'Precipitation_mm': round(precip_mm, 1)
            })
        else:
            monthly_data.append({
                'Year': year,
                'Month': month,
                'Month_Name': calendar.month_name[month],
                'Precipitation_mm': None
            })
    
    return monthly_data

def calculate_baseline_statistics(geometry, start_year, end_year):
    """Calculate baseline precipitation statistics"""
    print(f"Calculating baseline statistics ({start_year}-{end_year})...")
    baseline_data = {month: [] for month in range(4, 11)}
    
    for year in range(start_year, end_year + 1):
        year_data = extract_monthly_precipitation(geometry, year)
        for month_data in year_data:
            if month_data['Precipitation_mm'] is not None:
                baseline_data[month_data['Month']].append(month_data['Precipitation_mm'])
    
    # Calculate statistics
    baseline_stats = []
    for month in range(4, 11):
        if len(baseline_data[month]) > 0:
            values = np.array(baseline_data[month])
            baseline_stats.append({
                'Month': month,
                'Month_Name': calendar.month_name[month],
                'Mean_mm': round(np.mean(values), 1),
                'Median_mm': round(np.median(values), 1),
                'StdDev': round(np.std(values), 1),
                'P10': round(np.percentile(values, 10), 1),
                'P25': round(np.percentile(values, 25), 1),
                'P75': round(np.percentile(values, 75), 1),
                'P90': round(np.percentile(values, 90), 1)
            })
    
    return pd.DataFrame(baseline_stats)

def classify_precipitation(value, mean, p10, p25, p75, p90):
    """Classify precipitation relative to baseline"""
    if value <= p10:
        return 'Extremely Dry'
    elif value <= p25:
        return 'Dry'
    elif value <= p75:
        return 'Normal'
    elif value <= p90:
        return 'Wet'
    else:
        return 'Extremely Wet'

In [ ]:
# Calculate baseline statistics
if aoi_geometry:
    print("Calculating precipitation baseline statistics...")
    baseline_df = calculate_baseline_statistics(
        aoi_geometry, 
        precip_baseline_start.value, 
        precip_baseline_end.value
    )
    
    print("\nBaseline Statistics (April-October):")
    display(baseline_df)
else:
    print("⚠ Please upload an AOI first")

In [ ]:
# Extract precipitation for analysis years
if aoi_geometry:
    print(f"Extracting precipitation data ({precip_analysis_start.value}-{precip_analysis_end.value})...")
    
    all_precip_data = []
    yearly_summaries = []
    
    for year in range(precip_analysis_start.value, precip_analysis_end.value + 1):
        year_data = extract_monthly_precipitation(aoi_geometry, year)
        
        # Add classification based on baseline
        for month_data in year_data:
            month_num = month_data['Month']
            baseline_row = baseline_df[baseline_df['Month'] == month_num]
            
            if not baseline_row.empty and month_data['Precipitation_mm'] is not None:
                baseline = baseline_row.iloc[0]
                month_data['Normal_mm'] = baseline['Mean_mm']
                month_data['Anomaly_mm'] = round(month_data['Precipitation_mm'] - baseline['Mean_mm'], 1)
                month_data['Anomaly_pct'] = round((month_data['Anomaly_mm'] / baseline['Mean_mm']) * 100, 1)
                month_data['Classification'] = classify_precipitation(
                    month_data['Precipitation_mm'],
                    baseline['Mean_mm'],
                    baseline['P10'],
                    baseline['P25'],
                    baseline['P75'],
                    baseline['P90']
                )
            else:
                month_data['Normal_mm'] = None
                month_data['Anomaly_mm'] = None
                month_data['Anomaly_pct'] = None
                month_data['Classification'] = 'No Data'
        
        all_precip_data.extend(year_data)
        
        # Calculate yearly summary
        valid_months = [m for m in year_data if m['Precipitation_mm'] is not None]
        if valid_months:
            total_precip = sum(m['Precipitation_mm'] for m in valid_months)
            total_normal = sum(m['Normal_mm'] for m in valid_months if m['Normal_mm'] is not None)
            
            if total_normal > 0:
                anomaly_pct = ((total_precip - total_normal) / total_normal) * 100
            else:
                anomaly_pct = 0
            
            # Classify season
            if anomaly_pct <= -30:
                season_class = 'Extremely Dry Season'
            elif anomaly_pct <= -15:
                season_class = 'Dry Season'
            elif anomaly_pct <= 15:
                season_class = 'Normal Season'
            elif anomaly_pct <= 30:
                season_class = 'Wet Season'
            else:
                season_class = 'Extremely Wet Season'
            
            yearly_summaries.append({
                'Year': year,
                'Total_Precip_mm': round(total_precip, 1),
                'Normal_mm': round(total_normal, 1),
                'Anomaly_mm': round(total_precip - total_normal, 1),
                'Anomaly_pct': round(anomaly_pct, 1),
                'Classification': season_class
            })
    
    precip_df = pd.DataFrame(all_precip_data)
    yearly_df = pd.DataFrame(yearly_summaries)
    
    print("\nYearly Precipitation Summary:")
    display(yearly_df)
else:
    print("⚠ Please upload an AOI first")

## 6. Visualization

In [ ]:
# Create comprehensive visualization
if aoi_geometry and 'yearly_df' in locals():
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle('Background Environmental Data Analysis', fontsize=16, fontweight='bold')
    
    # 1. Precipitation anomaly chart
    ax1 = axes[0, 0]
    colors = ['brown' if a < 0 else 'lightblue' for a in yearly_df['Anomaly_pct']]
    bars = ax1.bar(yearly_df['Year'], yearly_df['Anomaly_pct'], color=colors, alpha=0.7)
    ax1.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    ax1.axhline(y=-15, color='orange', linestyle='--', alpha=0.5, label='Dry threshold')
    ax1.axhline(y=15, color='blue', linestyle='--', alpha=0.5, label='Wet threshold')
    ax1.set_xlabel('Year')
    ax1.set_ylabel('Precipitation Anomaly (%)')
    ax1.set_title('Growing Season Precipitation Anomaly')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Add classification labels
    for i, (year, classification) in enumerate(zip(yearly_df['Year'], yearly_df['Classification'])):
        ax1.text(year, yearly_df['Anomaly_pct'].iloc[i] + 2, 
                classification.split()[0], ha='center', fontsize=8, rotation=45)
    
    # 2. Total precipitation by year
    ax2 = axes[0, 1]
    ax2.bar(yearly_df['Year'], yearly_df['Total_Precip_mm'], color='steelblue', alpha=0.7)
    if len(yearly_df) > 0:
        ax2.axhline(y=yearly_df['Normal_mm'].iloc[0], color='red', linestyle='--', 
                   label=f"Normal ({yearly_df['Normal_mm'].iloc[0]:.0f} mm)")
    ax2.set_xlabel('Year')
    ax2.set_ylabel('Precipitation (mm)')
    ax2.set_title('Total Growing Season Precipitation')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # 3. Crop rotation timeline
    ax3 = axes[1, 0]
    if 'aafc_df' in locals() and len(aafc_df) > 0:
        unique_crops = aafc_df['Primary_Crop'].unique()
        colors = plt.cm.Set3(np.linspace(0, 1, len(unique_crops)))
        crop_colors = dict(zip(unique_crops, colors))
        
        bars = ax3.bar(aafc_df['Year'], [1]*len(aafc_df), 
                      color=[crop_colors[crop] for crop in aafc_df['Primary_Crop']])
        
        for i, (year, crop) in enumerate(zip(aafc_df['Year'], aafc_df['Primary_Crop'])):
            ax3.text(year, 0.5, crop[:10], ha='center', va='center', rotation=90, fontsize=8)
        
        ax3.set_xlabel('Year')
        ax3.set_title('Crop Rotation History')
        ax3.set_ylim(0, 1)
        ax3.set_yticks([])
    
    # 4. Summary table
    ax4 = axes[1, 1]
    ax4.axis('off')
    
    # Combine AAFC and precipitation data
    if 'aafc_df' in locals():
        summary_data = []
        for year in yearly_df['Year']:
            crop_row = aafc_df[aafc_df['Year'] == year]
            precip_row = yearly_df[yearly_df['Year'] == year]
            
            if not crop_row.empty and not precip_row.empty:
                summary_data.append([
                    year,
                    crop_row.iloc[0]['Primary_Crop'][:15],
                    f"{precip_row.iloc[0]['Anomaly_pct']:.0f}%",
                    precip_row.iloc[0]['Classification'].split()[0]
                ])
        
        if summary_data:
            table = ax4.table(cellText=summary_data,
                            colLabels=['Year', 'Crop', 'Precip Anomaly', 'Classification'],
                            cellLoc='center',
                            loc='center')
            table.auto_set_font_size(False)
            table.set_fontsize(9)
            table.scale(1, 1.5)
    
    ax4.set_title('Combined Summary', fontweight='bold')
    
    plt.tight_layout()
    plt.savefig('background_data_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\n✓ Figure saved as 'background_data_analysis.png'")
else:
    print("⚠ Run data extraction first")

## 7. Export Combined Dataset

In [ ]:
# Combine all data into export format
if aoi_geometry and 'aafc_df' in locals() and 'yearly_df' in locals():
    
    # Create combined dataset
    export_data = []
    
    for year in yearly_df['Year']:
        # Get crop data
        crop_row = aafc_df[aafc_df['Year'] == year]
        # Get precipitation data
        precip_row = yearly_df[yearly_df['Year'] == year]
        # Get monthly precipitation
        monthly_precip = precip_df[precip_df['Year'] == year]
        
        row_data = {
            'Year': year,
            'AOI_Name': uploaded_filename if uploaded_filename else 'Unknown',
        }
        
        # Add crop data
        if not crop_row.empty:
            row_data.update({
                'Primary_Crop': crop_row.iloc[0]['Primary_Crop'],
                'Primary_Crop_Code': crop_row.iloc[0]['Primary_Crop_Code'],
                'Primary_Crop_Pct': crop_row.iloc[0]['Primary_Crop_Pct'],
                'Secondary_Crop': crop_row.iloc[0]['Secondary_Crop']
            })
        else:
            row_data.update({
                'Primary_Crop': None,
                'Primary_Crop_Code': None,
                'Primary_Crop_Pct': None,
                'Secondary_Crop': None
            })
        
        # Add monthly precipitation
        for month in range(4, 11):
            month_data = monthly_precip[monthly_precip['Month'] == month]
            if not month_data.empty:
                month_name = calendar.month_name[month][:3]
                row_data[f'{month_name}_Precip_mm'] = month_data.iloc[0]['Precipitation_mm']
                row_data[f'{month_name}_Anomaly_pct'] = month_data.iloc[0].get('Anomaly_pct', None)
            else:
                month_name = calendar.month_name[month][:3]
                row_data[f'{month_name}_Precip_mm'] = None
                row_data[f'{month_name}_Anomaly_pct'] = None
        
        # Add seasonal summary
        if not precip_row.empty:
            row_data.update({
                'Season_Total_Precip_mm': precip_row.iloc[0]['Total_Precip_mm'],
                'Season_Normal_mm': precip_row.iloc[0]['Normal_mm'],
                'Season_Anomaly_mm': precip_row.iloc[0]['Anomaly_mm'],
                'Season_Anomaly_pct': precip_row.iloc[0]['Anomaly_pct'],
                'Season_Classification': precip_row.iloc[0]['Classification']
            })
        else:
            row_data.update({
                'Season_Total_Precip_mm': None,
                'Season_Normal_mm': None,
                'Season_Anomaly_mm': None,
                'Season_Anomaly_pct': None,
                'Season_Classification': None
            })
        
        export_data.append(row_data)
    
    # Create DataFrame
    export_df = pd.DataFrame(export_data)
    
    # Save to CSV
    output_filename = f"background_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    export_df.to_csv(output_filename, index=False)
    
    print(f"✓ Data exported to: {output_filename}")
    print(f"\nExport contains {len(export_df)} years of data with {len(export_df.columns)} columns")
    print("\nColumns included:")
    print(" - Crop data (AAFC)")
    print(" - Monthly precipitation (April-October)")
    print(" - Seasonal totals and anomalies")
    print(" - Classifications (Dry/Normal/Wet)")
    
    # Display preview
    print("\nData Preview:")
    display(export_df.head())
    
    # Also save metadata
    metadata = {
        'aoi_file': uploaded_filename,
        'aafc_years': f"{aafc_start.value}-{aafc_end.value}",
        'precip_baseline': f"{precip_baseline_start.value}-{precip_baseline_end.value}",
        'precip_analysis': f"{precip_analysis_start.value}-{precip_analysis_end.value}",
        'export_date': datetime.now().isoformat(),
        'n_years': len(export_df),
        'columns': list(export_df.columns)
    }
    
    metadata_filename = output_filename.replace('.csv', '_metadata.json')
    with open(metadata_filename, 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\n✓ Metadata saved to: {metadata_filename}")
    
else:
    print("⚠ Please complete data extraction before exporting")

## Summary

This notebook has extracted and combined:
1. **AAFC Crop History** - Annual crop types for your AOI
2. **ERA-5 Precipitation** - Monthly precipitation with anomaly analysis
3. **Classifications** - Dry/Normal/Wet year categorization

The exported CSV file can now be used in your reclamation analysis to:
- Account for crop rotation patterns
- Normalize performance metrics for weather conditions
- Identify years with abnormal precipitation that may affect vegetation
- Provide context for interpreting reclamation success